In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import cv2
from google.colab.patches import cv2_imshow

from tqdm import tqdm

import os
import warnings
warnings.filterwarnings(action='ignore')



In [ ]:
## 이미지 파일 불러오는 함수

# 세부 디렉토리 이름 가져오기
def load_image_path(dir):
    food_middle_list = sorted(os.listdir(dir))
    food_list = []  # 음식 이름
    food_path_list = []  # 음식 이름 경로

    for food_middle in food_middle_list:
        middle_path = os.path.join(dir, food_middle)
        food_name_list = os.listdir(middle_path)

        for food in food_name_list:
            path = os.path.join(middle_path, food)
            food_path_list.append(path)
            name = path.split('/')[-1]
            food_list.append(name)

    return food_list, food_path_list

# 세부 디렉토리의 이미지 파일 이름 가져오기
def get_all_image_files(dir):
    img_extensions = {'.jpg', '.jpeg', '.png', '.bmp'}
    image_files = []

    for root, _, files in os.walk(dir):
        for file in files:
            if any(file.lower().endswith(ext) for ext in img_extensions):
                image_files.append(os.path.join(root, file))

    return image_files

In [ ]:
raw_dir = '/content/drive/MyDrive/project3/data/traindata/train_image_crop (1)'
food_list, food_path_list = load_image_path(raw_dir)
image_path = get_all_image_files(food_path_list[0])

print(len(food_list))
print(food_list)
print(food_path_list)
print(image_path[:5])

4
['갈비구이', '곱창구이', '고등어구이', '갈치구이']
['/content/drive/MyDrive/project3/data/traindata/train_image_crop (1)/구이/갈비구이', '/content/drive/MyDrive/project3/data/traindata/train_image_crop (1)/구이/곱창구이', '/content/drive/MyDrive/project3/data/traindata/train_image_crop (1)/구이/고등어구이', '/content/drive/MyDrive/project3/data/traindata/train_image_crop (1)/구이/갈치구이']
['/content/drive/MyDrive/project3/data/traindata/train_image_crop (1)/구이/갈비구이/Img_000_0460.jpg', '/content/drive/MyDrive/project3/data/traindata/train_image_crop (1)/구이/갈비구이/Img_000_0764.jpg', '/content/drive/MyDrive/project3/data/traindata/train_image_crop (1)/구이/갈비구이/Img_000_0970_crop.jpg', '/content/drive/MyDrive/project3/data/traindata/train_image_crop (1)/구이/갈비구이/Img_000_0820.jpg', '/content/drive/MyDrive/project3/data/traindata/train_image_crop (1)/구이/갈비구이/Img_000_0005.jpg']


In [ ]:
# histogram 그리기
def calculate_histogram(image):
    # Gray Scale 일떄
    if len(image.shape) == 2:
        histogram = cv2.calcHist([image], # 이미지
                                 [0], # 사용될 채널
                                 None, # 마스크
                                 [256], #bins
                                 [0, 256]) #각 채널의 최소값 / 최대값
    else:  # 컬러 이미지
        histogram = cv2.calcHist([image],
                                 [0, 1, 2],
                                 None,
                                 [256, 256, 256],
                                 [0, 256, 0, 256, 0, 256])

    # 히스토그램 정규화
    cv2.normalize(histogram, histogram)
    return histogram


# 원본과 가장 비슷한 hist를 갖는 그림을 선정
def find_best_match(image, target_hist, patch_size):

    # 가장 비슷한 사이즈를 정하기 위한 준비
    min_diff = float('inf')
    best_x, best_y = 0, 0

    # 이미지 크기
    # Gray Scale 일떄
    if len(image.shape) == 2:
        height, width = image.shape
    # 컬러 이미지
    else:
        height, width, _ = image.shape
    # patch의 위치를 20씩 이동시키며 비교
    for y in range(0, height - patch_size, 20):
        for x in range(0, width - patch_size, 20):
            patch = image[y:y + patch_size, x:x + patch_size]
            patch_hist = calculate_histogram(patch)
            diff = cv2.compareHist(target_hist, patch_hist, cv2.HISTCMP_BHATTACHARYYA)
            if diff < min_diff:
                min_diff = diff
                best_x, best_y = x, y

    return best_x, best_y

def extract_best_patch(image):

    image_size_adj = 20

    # gray로 판별하는 방법
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    height_g, width_g = image_gray.shape
    patch_size_g = min(height_g - image_size_adj, width_g - image_size_adj)  # 1:1 비율의 패치 크기 결정(원본보다 20 작게)
    target_hist_g = calculate_histogram(image_gray)

    best_x_g, best_y_g = find_best_match(image_gray, target_hist_g, patch_size_g)
    best_patch_gray = image[best_y_g:best_y_g + patch_size_g, best_x_g:best_x_g + patch_size_g]


    # color로 판별
    height, width, _ = image.shape

    patch_size = min(height -image_size_adj, width - image_size_adj)  # 1:1 비율의 패치 크기 결정(원본보다 20 작게)
    target_hist = calculate_histogram(image)  # 원본 이미지의 히스토그램 계산

    best_x, best_y = find_best_match(image, target_hist, patch_size)
    best_patch_color = image[best_y:best_y + patch_size, best_x:best_x + patch_size]

    return best_patch_gray, best_patch_color

In [ ]:
# 최선의 그림을 찾아서 저장하기
def select_best_patch_image(raw_image_path):

    # 디렉토리 생성
    color_image_path = raw_image_path.replace('train_image_crop (1)', 'selected_image_color')
    if not os.path.exists(color_image_path):
        os.makedirs(color_image_path)

    gray_image_path = raw_image_path.replace('train_image_crop (1)', 'selected_image_gray')
    if not os.path.exists(gray_image_path):
        os.makedirs(gray_image_path)

    images_path = get_all_image_files(raw_image_path)
    print(raw_image_path)
    for image_path in tqdm(images_path):
        img = cv2.imread(image_path)
        if img is not None:

            best_patch_gray = extract_best_patch(img)[0]
            best_patch_color = extract_best_patch(img)[1]

            # extract_best_patch(img[0]) -> gray hist로 선택
            # gray_select 만들기
            output_gray_image = best_patch_gray
            output_gray_image_path = image_path.replace('train_image_crop (1)', 'selected_image_gray').replace('Img', 'select_gray')

            # 저장하기
            cv2.imwrite(output_gray_image_path, output_gray_image)

            # extract_best_patch(img[1]) -> color hist로 선택
            # color_select 만들기
            output_color_image = best_patch_color
            output_color_image_path = image_path.replace('train_image_crop (1)', 'selected_image_color').replace('Img', 'select_color')

            # color 저장하기
            cv2.imwrite(output_color_image_path, output_color_image)

In [ ]:
raw_dir = '/content/drive/MyDrive/project3/data/traindata/train_image_crop (1)'
food_list, food_path_list = load_image_path(raw_dir)
for food_path in food_path_list:
  select_best_patch_image(food_path)

/content/drive/MyDrive/project3/data/traindata/train_image_crop (1)/구이/갈비구이


100%|██████████| 996/996 [28:52<00:00,  1.74s/it]


/content/drive/MyDrive/project3/data/traindata/train_image_crop (1)/구이/곱창구이


100%|██████████| 1000/1000 [27:40<00:00,  1.66s/it]


/content/drive/MyDrive/project3/data/traindata/train_image_crop (1)/구이/고등어구이


 12%|█▎        | 125/1000 [04:09<29:06,  2.00s/it]


KeyboardInterrupt: 